# Tablas Particionadas

## Crear base de Datos

In [38]:
USE master;
GO

IF NOT EXISTS (
		SELECT 1
		FROM sys.databases
		WHERE name = 'PartitionedData'
		)
BEGIN
	CREATE DATABASE PartitionedData ON (
		NAME = PartitionedData_DATA
		, FILENAME = '/var/opt/mssql/data/PartitionedData_DATA.mdf'
		, SIZE = 10
		, MAXSIZE = 500 MB
		, FILEGROWTH = 5
		) LOG ON (
		NAME = PartitionedData_LOG
		, FILENAME = '/var/opt/mssql/data/PartitionedData_LOG.ldf'
		, SIZE = 5 MB
		, MAXSIZE = 1 GB
		, FILEGROWTH = 5 MB
		);
END

USE PartitionedData


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020

In [39]:
CREATE TABLE dbo.SalesOrder (
	SalesOrderID INT IDENTITY(1, 1) CONSTRAINT PK_SalesOrder_SalesOrderID PRIMARY KEY
	, SalesUniqueID UNIQUEIDENTIFIER CONSTRAINT DF_SalesOrder_SalesUniqueID DEFAULT NEWID()
	, OrderDate DATE
	, SellerID INT
    , CompanyID INT
	)


Commands completed successfully.

Total execution time: 00:00:00.004

In [ ]:
DECLARE @CompanyFoundedDate DATE = '2009-02-10'
DECLARE @DaysToToday INT = DATEDIFF(DAY, @CompanyFoundedDate, GetDate())
DECLARE @LoopStep INT = 1
		,@LoopTotal int = 10000

WHILE @LoopStep <= @LoopTotal
BEGIN
INSERT INTO dbo.SalesOrder (
	  OrderDate
	, SellerID
	, CompanyID
	)
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
UNION
SELECT DATEADD(DAY, ABS(CHECKSUM(NewId())) % @DaysToToday, @CompanyFoundedDate)
	, ABS(CHECKSUM(NewId())) % 25
	, ABS(CHECKSUM(NewId())) % 1000
	set @LoopStep = @LoopStep +1
END

In [1]:
Select count(1) from salesorder

(1 row affected)

Total execution time: 00:00:00.067

(No column name)
100000


In [ ]:
set statistics io ON

SELECT * FROM SalesOrder where SellerID = 12 and OrderDate BETWEEN '2014-01-01' and '2014-12-31'

In [2]:
ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2009

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2010

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2011

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2012

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2013

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2014

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2015

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2016

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2017

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2018

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2019

ALTER DATABASE PartitionedData
ADD FILEGROUP Sales2020



Commands completed successfully.

Total execution time: 00:00:00.019

In [3]:
SELECT name AS AvailableFilegroups
FROM sys.filegroups
WHERE type = 'FG'

(13 rows affected)

Total execution time: 00:00:00.011

AvailableFilegroups
PRIMARY
Sales2009
Sales2010
Sales2011
Sales2012
Sales2013
Sales2014
Sales2015
Sales2016
Sales2017


In [4]:
DECLARE @FilegroupName SYSNAME
	, @FilesPath VARCHAR(200) = '/var/opt/mssql/data/'

DECLARE FileCreation CURSOR
FOR
(
		SELECT name AS AvailableFilegroups
		FROM sys.filegroups
		WHERE type = 'FG'
			AND name <> 'Primary'
		)

OPEN FileCreation

FETCH NEXT
FROM FileCreation
INTO @FileGroupName

WHILE @@FETCH_STATUS = 0
BEGIN
	DECLARE @CMD NVARCHAR(MAX) = CONCAT (
			'ALTER DATABASE PartitionedData
ADD FILE
(NAME = [Partition'
			, @FileGroupName
			, ']
,FILENAME = '''
			, @FilesPath
			, 'Partition'
			, @FileGroupName
			, '.ndf'') TO FILEGROUP [',@FileGroupName,']'
			)

	EXEC SP_EXECUTESQL @Command1 = @CMD

	FETCH NEXT
	FROM FileCreation
	INTO @FileGroupName
END


Commands completed successfully.

Total execution time: 00:00:05.096

In [5]:
sp_helpdb PartitionedData

Total execution time: 00:00:00.066

name,db_size,owner,dbid,created,status,compatibility_level
PartitionedData,166.00 MB,sa,10,Oct 7 2020,"Status=ONLINE, Updateability=READ_WRITE, UserAccess=MULTI_USER, Recovery=FULL, Version=904, Collation=SQL_Latin1_General_CP1_CI_AS, SQLSortOrder=52, IsAutoCreateStatistics, IsAutoUpdateStatistics, IsFullTextEnabled",150


name,fileid,filename,filegroup,size,maxsize,growth,usage
PartitionedData_DATA,1,/var/opt/mssql/data/PartitionedData_DATA.mdf,PRIMARY,46080 KB,51200 KB,5120 KB,data only
PartitionedData_LOG,2,/var/opt/mssql/data/PartitionedData_LOG.ldf,NULL,25600 KB,25600 KB,5120 KB,log only
PartitionSales2009,3,/var/opt/mssql/data/PartitionSales2009.ndf,Sales2009,8192 KB,Unlimited,65536 KB,data only
PartitionSales2010,4,/var/opt/mssql/data/PartitionSales2010.ndf,Sales2010,8192 KB,Unlimited,65536 KB,data only
PartitionSales2011,5,/var/opt/mssql/data/PartitionSales2011.ndf,Sales2011,8192 KB,Unlimited,65536 KB,data only
PartitionSales2012,6,/var/opt/mssql/data/PartitionSales2012.ndf,Sales2012,8192 KB,Unlimited,65536 KB,data only
PartitionSales2013,7,/var/opt/mssql/data/PartitionSales2013.ndf,Sales2013,8192 KB,Unlimited,65536 KB,data only
PartitionSales2014,8,/var/opt/mssql/data/PartitionSales2014.ndf,Sales2014,8192 KB,Unlimited,65536 KB,data only
PartitionSales2015,9,/var/opt/mssql/data/PartitionSales2015.ndf,Sales2015,8192 KB,Unlimited,65536 KB,data only
PartitionSales2016,10,/var/opt/mssql/data/PartitionSales2016.ndf,Sales2016,8192 KB,Unlimited,65536 KB,data only


In [6]:
CREATE PARTITION FUNCTION [SalesPartitionByYear] (Date)
AS RANGE LEFT FOR VALUES ('20090101','20100101','20110101','20120101','20130101','20140101','20150101','20160101','20170101','20180101','20190101','20200101')

Commands completed successfully.

Total execution time: 00:00:00.004

In [7]:
CREATE PARTITION SCHEME SalesDataScheme
as PARTITION SalesPartitionByYear
TO (Sales2009,Sales2010,Sales2011,Sales2012,Sales2013,Sales2014,Sales2015,Sales2016,Sales2017,Sales2018,Sales2019,Sales2020,[PRIMARY])

Commands completed successfully.

Total execution time: 00:00:00.003

Commands completed successfully.

Total execution time: 00:00:00.004

In [8]:
CREATE TABLE dbo.Sales (
	SalesID INT IDENTITY(1, 1) NOT NULL
	, SalesUniqueID UNIQUEIDENTIFIER CONSTRAINT DF_Sales_SalesUniqueID DEFAULT NEWID()
	, OrderDate DATE NOT NULL
	, SellerID INT
    , CompanyID INT
    , CONSTRAINT PK_Sales_SalesOrderID PRIMARY KEY (SalesID, OrderDate)
	) ON SalesDataScheme(OrderDate)

Commands completed successfully.

Total execution time: 00:00:00.006

In [10]:
SET IDENTITY_INSERT dbo.Sales ON

INSERT INTO dbo.Sales (
	SalesID
	, SalesUniqueID
	, OrderDate
	, SellerID
	, CompanyID
	)
SELECT SalesOrderID
	, SalesUniqueID
	, OrderDate
	, SellerID
	, CompanyID
FROM SalesOrder


: Msg 9002, Level 17, State 4, Line 3
The transaction log for database 'PartitionedData' is full due to 'ACTIVE_TRANSACTION'.

: Msg 9002, Level 17, State 4, Line 3
The transaction log for database 'PartitionedData' is full due to 'ACTIVE_TRANSACTION'.

Total execution time: 00:00:02.718

Commands completed successfully.

Total execution time: 00:00:00.438